In [ ]:
import torch.nn as nn
import torch
import MST
import numpy as np

In [ ]:
np.random.seed(42)

In [ ]:
in_C = 1
out_C = 1
stride = 1
padding = 1
dilation = 1
kernel = 3

params = [in_C, out_C, kernel, stride, padding, dilation]

print(f"{(16 - kernel + padding*2 - (kernel-1)*(dilation-1))/stride + 1}")

In [ ]:
with torch.no_grad():
    myConv = MST.Conv2d(*params)
    torchConv = nn.Conv2d(*params)

    torchConv.weight = nn.Parameter(torch.tensor(myConv._w))
    torchConv.bias = nn.Parameter(torch.tensor(myConv._bias).flatten())


In [ ]:
image = np.ones((1, in_C, 16, 16))

In [ ]:
with torch.no_grad():
    myres = myConv(image)
    nnres = torchConv(torch.Tensor(image))
    print(myres.shape)
    print(nnres.shape)

    nnres = nnres.numpy().round(3).flatten()
    myres = myres.round(3).flatten()
    bads = np.where(abs(nnres - myres) > 0.001)
    print(myres[:20])
    print(nnres[:20])
    print(len(bads[0]))
    for pos in bads[0][:10]:
        print(f"[{pos}]: {myres[pos]:.3f} {nnres[pos]:.3f}")

In [ ]:
%%timeit
myres = myConv(image)

In [ ]:
%%timeit
nnres = torchConv(torch.Tensor(image))

In [5]:
def to_np_arr(a):
    a_ = MST.MDT_ARRAY(a).astype(np.float32) / 255
    return a_.reshape(1, *a_.shape)

import torch.nn as nn
import torch
import MST
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets
np.random.seed(42)
in_C = 1
out_C = 6
stride = 1
padding = 0
dilation = 1
kernel = 5
inH, InW = 32, 32
outH, outW = inH//stride, InW//stride

params = [in_C, out_C, kernel, stride, padding, dilation]

print(f"{(16 - kernel + padding*2 - (kernel-1)*(dilation-1))/stride + 1}")
myConv = MST.Conv2d(*params)
torchConv = nn.Conv2d(*params)

torchConv.weight = nn.Parameter(torch.tensor(myConv._w))
torchConv.bias = nn.Parameter(torch.tensor(myConv._bias).flatten())
image = np.ones((1, in_C, inH, InW))

transform = to_np_arr

12.0


In [6]:
# torchImage = torch.ones((1, in_C, inH, InW), requires_grad=True)

from tqdm.auto import tqdm
train_dataset = datasets.MNIST(
    root='datasets',
    train=True,
    transform=transform,
    download=True
)


train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=0,
)

for (images, labels) in tqdm(train_dataloader):

    torchImage = images
    image = torchImage.detach().numpy()
    
    out_sample = myConv(image)

    dOut = np.random.random(out_sample.shape)

    myres = myConv(image)
    myres.backward(dOut)
    nnres = torchConv(torchImage)
    nnres.backward(torch.Tensor(dOut))

    # nngrad = torchImage.grad.numpy().round(3).flatten()
    # mygrad = myConv._dinX.round(3).flatten()
    # bads = np.where(abs(mygrad - nngrad) > 0.001)
    # print(len(bads[0]))
    # for pos in bads[0][:10]:
    #     print(f"[{pos}]: {mygrad[pos]:.3f} {nngrad[pos]:.3f}")

    nnres = nnres.detach().numpy().round(3).flatten()
    myres = myres.round(3).flatten()
    bads = np.where(abs(nnres - myres) > 0.001)
    print(len(bads[0]))
    for pos in bads[0][:10]:
        print(f"[{pos}]: {myres[pos]:.3f} {nnres[pos]:.3f}")

    nngrad = torchConv.weight.grad.numpy().round(3).flatten()
    mygrad = myConv._dw.round(3).flatten()
    bads = np.where(abs(mygrad - nngrad) > 0.001)
    print(len(bads[0]))
    for pos in bads[0][:10]:
        print(f"[{pos}]: {mygrad[pos]:.3f} {nngrad[pos]:.3f}")

    torchConv.weight.grad = torch.zeros_like(torchConv.weight.grad)

    nngrad = torchConv.bias.grad.numpy().round(3).flatten()
    mygrad = myConv._dbias.round(3).flatten()
    bads = np.where(abs(mygrad - nngrad) > 0.001)
    print(len(bads[0]))
    for pos in bads[0][:10]:
        print(f"[{pos}]: {mygrad[pos]:.3f} {nngrad[pos]:.3f}")

    torchConv.bias.grad = torch.zeros_like(torchConv.bias.grad)

  0%|          | 0/1875 [00:00<?, ?it/s]

0
150
[0]: 1550.044 1557.219
[1]: 1560.743 1566.263
[2]: 1560.553 1566.139
[3]: 1561.041 1576.387
[4]: 1564.901 1589.313
[5]: 1563.159 1568.157
[6]: 1576.323 1579.640
[7]: 1577.781 1577.171
[8]: 1577.572 1583.462
[9]: 1585.569 1591.306
5
[0]: 9234.310 9234.312
[1]: 9181.919 9181.913
[2]: 9163.641 9163.643
[3]: 9221.013 9221.016
[5]: 9226.397 9226.402
0
150
[0]: 1779.923 1788.718
[1]: 1770.438 1796.120
[2]: 1762.029 1796.636
[3]: 1768.360 1804.460
[4]: 1770.700 1797.782
[5]: 1800.136 1818.084
[6]: 1794.424 1819.393
[7]: 1798.328 1819.275
[8]: 1804.457 1825.720
[9]: 1793.948 1824.286
4
[0]: 9261.922 9261.917
[3]: 9191.053 9191.050
[4]: 9188.185 9188.179
[5]: 9265.790 9265.793
0
150
[0]: 1521.350 1509.140
[1]: 1521.139 1505.883
[2]: 1519.948 1509.389
[3]: 1512.867 1503.948
[4]: 1516.843 1486.252
[5]: 1545.832 1545.103
[6]: 1545.150 1551.304
[7]: 1548.709 1555.921
[8]: 1551.427 1539.136
[9]: 1548.124 1521.082
4
[1]: 9223.555 9223.551
[2]: 9264.668 9264.672
[4]: 9213.231 9213.232
[5]: 9170.

KeyboardInterrupt: 

In [3]:
with torch.no_grad():
    myres = myConv(image)
    nnres = torchConv(torch.Tensor(image))
    print(myres.shape)
    print(nnres.shape)

    nnres = nnres.numpy().round(3).flatten()
    myres = myres.round(3).flatten()
    bads = np.where(abs(nnres - myres) > 0.001)
    print(myres[:20])
    print(nnres[:20])
    print(len(bads[0]))
    for pos in bads[0][:10]:
        print(f"[{pos}]: {myres[pos]:.3f} {nnres[pos]:.3f}")

IndexError: index 30 is out of bounds for axis 2 with size 30

In [1]:
import torch.nn as nn
import torch
import MST
import numpy as np
np.random.seed(42)
BS = 5
in_C = 1
out_C = 6
stride = 1
padding = 1
dilation = 1
kernel = 5
inH, InW = 32, 32
outH, outW = inH//stride, InW//stride

params = [in_C, out_C, kernel, stride, padding, dilation]

# print(f"{(16 - kernel + padding*2 - (kernel-1)*(dilation-1))/stride + 1}")
myConv = MST.Conv2d(*params)
torchConv = nn.Conv2d(*params)

torchConv.weight = nn.Parameter(torch.tensor(myConv._w))
torchConv.bias = nn.Parameter(torch.tensor(myConv._bias).flatten())
image = np.ones((BS, in_C, inH, InW))
# torchImage = torch.rand((BS, in_C, inH, InW), requires_grad=True)
# image = torchImage.detach().numpy()

In [2]:
with torch.no_grad():
    myres = myConv(image)
    nnres = torchConv(torch.Tensor(image))
    print(myres.shape)
    print(nnres.shape)

    nnres = nnres.numpy().round(3).flatten()
    myres = myres.round(3).flatten()
    bads = np.where(abs(nnres - myres) > 0.001)
    print(myres[54:70])
    print(nnres[54:70])
    print(len(bads[0]))
    for pos in bads[0][:10]:
        print(f"[{pos}]: {myres[pos]:.3f} {nnres[pos]:.3f}")

(5, 6, 30, 30)
torch.Size([5, 6, 30, 30])
[-1.156 -1.156 -1.156 -1.156 -1.156 -0.202 -1.355 -1.156 -1.156 -1.156
 -1.156 -1.156 -1.156 -1.156 -1.156 -1.156]
[-1.156 -1.156 -1.156 -1.156 -1.156 -0.202 -1.355 -1.156 -1.156 -1.156
 -1.156 -1.156 -1.156 -1.156 -1.156 -1.156]
0


In [10]:


out_sample = myConv(image)

myConv(image).backward(np.ones(out_sample.shape))
torchConv(torchImage).backward(torch.ones(out_sample.shape))


nngrad = torchImage.grad.numpy().round(3).flatten()
mygrad = myConv._dinX.round(3).flatten()
bads = np.where(abs(mygrad - nngrad) > 0.001)
print(len(bads[0]))
for pos in bads[0][:10]:
    print(f"[{pos}]: {mygrad[pos]:.3f} {nngrad[pos]:.3f}")


nngrad = torchConv.weight.grad.numpy().round(3).flatten()
mygrad = myConv._dw.round(3).flatten()
bads = np.where(abs(mygrad - nngrad) > 0.001)
print(len(bads[0]))
for pos in bads[0][:10]:
    print(f"[{pos}]: {mygrad[pos]:.3f} {nngrad[pos]:.3f}")

torchConv.weight.grad = torch.zeros_like(torchConv.weight.grad)

nngrad = torchConv.bias.grad.numpy().round(3).flatten()
mygrad = myConv._dbias.round(3).flatten()
bads = np.where(abs(mygrad - nngrad) > 0.001)
print(len(bads[0]))
for pos in bads[0][:10]:
    print(f"[{pos}]: {mygrad[pos]:.3f} {nngrad[pos]:.3f}")

torchConv.bias.grad = torch.zeros_like(torchConv.bias.grad)

0
0
0
